<hr style="height:2px;">

# Demo: Apply trained CARE model for joint denoising and surface projection of *Drosophila melanogaster* wing

This notebook demonstrates applying a CARE model for a denoising+projection task, assuming that training was already completed via [2_training.ipynb](2_training.ipynb).  
The trained model is assumed to be located in the folder `models` with the name `my_model`.

**Note:** The CARE model is here applied to an image that the model was also trained on.  
Of course, in practice one would typically use it to restore images that the model hasn't seen during training.

More documentation is available at http://csbdeep.bioimagecomputing.com/doc/.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import ProjectionCARE

<hr style="height:2px;">

# Download example data

The example data (also for testing) should have been downloaded in [1_datagen.ipynb](1_datagen.ipynb).  
Just in case, we will download it here again if it's not already present.

In [ ]:
download_and_extract_zip_file (
    url       = 'http://csbdeep.bioimagecomputing.com/example_data/flywing.zip',
    targetdir = 'data',
)

<hr style="height:2px;">

# Raw low-SNR 3D stack and associated high-SNR 2D surface projection

Plot the image pair and define its image axes, which will be needed later for CARE prediction.

In [ ]:
y = imread('data/flywing/GT/session_7_P17.tif')
x = imread('data/flywing/low_C3/session_7_P17.tif')

axes = 'ZYX'
print('source image size =', x.shape)
print('source image axes =', axes)
print('target image size =', y.shape)

plt.figure(figsize=(16,10))
plot_some(np.stack([x,np.broadcast_to(y,x.shape)]),
          title_list=[['low-SNR stack (maximum projection)','high-SNR (surface extracted with PreMosa)']], 
          pmin=2,pmax=99.8);

<hr style="height:2px;">

# CARE model

Load trained model (located in base directory `models` with name `my_model`) from disk.  
The configuration was saved during training and is automatically loaded when `ProjectionCARE` is initialized with `config=None`.

In [ ]:
model = ProjectionCARE(config=None, name='my_model', basedir='models')

## Apply CARE network to raw image

Predict the restored image (image will be successively split into smaller tiles if there are memory issues).

In [ ]:
%%time
restored = model.predict(x, axes)

Alternatively, one can directly set `n_tiles` to avoid the time overhead from multiple retries in case of memory issues.

**Note**: *Out of memory* problems during `model.predict` can also indicate that the GPU is used by another process. In particular, shut down the training notebook before running the prediction (you may need to restart this notebook).

In [ ]:
%%time
restored = model.predict(x, axes, n_tiles=(1,4,4))

## Save restored image

Save the restored image stack as a ImageJ-compatible TIFF image, i.e. the image can be opened in ImageJ/Fiji with correct axes semantics.

In [ ]:
Path('results').mkdir(exist_ok=True)
axes_restored = axes.replace(model.proj_params.axis,'')
save_tiff_imagej_compatible('results/%s_denoising+projection_example.tif' % model.name, restored, axes_restored)

<hr style="height:2px;">

# Raw low/high-SNR pair and denoised+projected image via CARE network

Plot the test stack pair and the predicted restored stack (middle).

In [ ]:
plt.figure(figsize=(16,10))
plot_some(np.stack([x,np.broadcast_to(restored,x.shape),np.broadcast_to(y,x.shape)]),
          title_list=[['low-SNR stack (maximum projection)','CARE','high-SNR (surface extracted with PreMosa)']], 
          pmin=2,pmax=99.8);